In [1]:
import re
import tabula
import requests
import json
import csv
import boto3
import pandas as pd
import numpy as np
from data_extraction import DataExtractor as de
from database_utils import DatabaseConnector as dc
from datetime import datetime as dt

extr = de()
conn = dc()

In [104]:
lengths = set()
for i in range(len(df)):
    lengths.add(df.loc[i, 'removed'])
print(lengths)

{'Removed', 'Still_avaliable'}


In [13]:
df = extr.read_rds_table(conn, conn.list_db_tables()[2])
df = df.drop(['level_0', 'first_name', 'last_name', '1'], axis=1)

display(df.head())

,date_uuid,user_uuid,card_number,store_code,product_code,product_quantity
index,,,,,,
0,9476f17e-5d6a-4117-874d-9cdb38ca1fa6,93caf182-e4e9-4c6e-bebb-60a1a9dcf9b8,30060773296197,BL-8387506C,R7-3126933h,3
1,0423a395-a04d-4e4a-bd0f-d237cbd5a295,8fe96c3a-d62d-4eb5-b313-cf12d9126a49,349624180933183,WEB-1388012W,C2-7287916l,2
2,65187294-bb16-4519-adc0-787bbe423970,fc461df4-b919-48b2-909e-55c95a03fe6b,3529023891650490,CH-01D85C8D,S7-1175877v,2
3,579e21f7-13cb-436b-83ad-33687a4eb337,6104719f-ef14-4b09-bf04-fb0c4620acb0,213142929492281,CL-C183BE4B,D8-8421505n,2
4,00ab86c3-2039-4674-b9c1-adbcbbf525bd,9523a6d3-b2dd-4670-a51a-36aebc89f579,502067329974,SO-B5B9CB3B,B6-2596063a,2


In [12]:
dg = extr.retrieve_store_data()
dg['continent'] = dg['continent'].replace({'eeEurope':'Europe'})
dg['continent'] = dg['continent'].replace({'eeAmerica':'America'})
dg = dg[dg['continent'].isin(['Europe', 'America'])]
dg['staff_numbers'] = dg['staff_numbers'].str.replace('\D', '', regex=True)
dg.drop(['index', 'lat'], axis=1, inplace=True)                            
dg.reset_index(drop=True, inplace=True)
dg['opening_date'] = pd.to_datetime(dg['opening_date']).dt.date

display(dg.head())

,address,longitude,locality,store_code,staff_numbers,opening_date,store_type,latitude,country_code,continent
0,N/A,N/A,N/A,WEB-1388012W,325,2010-06-12,Web Portal,None,GB,Europe
1,"Flat 72W\nSally isle\nEast Deantown\nE7B 8EB, ...",51.62907,High Wycombe,HI-9B97EE4E,34,1996-10-25,Local,-0.74934,GB,Europe
2,"Heckerstraße 4/5\n50491 Säckingen, Landshut",48.52961,Landshut,LA-0772C7B9,92,2013-04-12,Super Store,12.16179,DE,Europe
3,"5 Harrison tunnel\nSouth Lydia\nWC9 2BE, Westbury",51.26,Westbury,WE-1DE82CEE,69,2014-01-02,Super Store,-2.1875,GB,Europe
4,Studio 6\nStephen landing\nSouth Simon\nB77 2W...,53.0233,Belper,BE-18074576,35,2019-09-09,Local,-1.48119,GB,Europe


In [14]:
uuids = set()
for uuid in df.loc[:, 'store_code']:
    if uuid in dg.loc[:, 'store_code']:
        uuids.add(uuid)
print(len(uuids))

0
